1. Install ElasticSearch (ES).
2. Install an ES plugin for Polish https://github.com/allegro/elasticsearch-analysis-morfologik 
3. Define an ES analyzer for Polish texts containing:
   1. standard tokenizer
   1. synonym filter with the following definitions:
      1. kpk - kodeks postępowania karnego
      1. kpc - kodeks postępowania cywilnego
      1. kk - kodeks karny
      1. kc - kodeks cywilny
   1. Morfologik-based lemmatizer
   1. lowercase filter

In [537]:
import re
import tarfile
import os
import matplotlib.pyplot as plt
import numpy as np
import regex
import pandas as pd
import morfeusz2
%matplotlib inline



from elasticsearch import Elasticsearch
from elasticsearch_dsl import analyzer, tokenizer

es = Elasticsearch("http://localhost:9200")
resp = es.info()
resp

{'name': 'x-dell',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'hrJwCHZtSGm_1kBHAztvag',
 'version': {'number': '7.10.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa',
  'build_date': '2020-12-05T01:00:33.671820Z',
  'build_snapshot': False,
  'lucene_version': '8.7.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [593]:
from elasticsearch_dsl import  *
from elasticsearch import *
from datetime import datetime

from elasticsearch_dsl import connections

connections.create_connection(hosts=['localhost'])
es = Elasticsearch()

synonym = token_filter('my_name', type='synonym',
                       synonyms =
                       ["kpk => kodeks postępowania karnego",
                        "kpc => kodeks postępowania cywilnego",
                        "kk => kodeks karny",
                        "kc => kodeks cywilny"
                       ])

my_analyzer = analyzer('my_analyzer',
    tokenizer=tokenizer('standard'),
    filter=["lowercase",synonym,"morfologik_stem",]
)


# es.delete_by_query(index=['ustawa'], body={"query": {"match_all": {}}})


class Ustawa(Document):
    title = Text()
    created_at = Integer()
    content = Text(analyzer=my_analyzer)
    
    class Index:
        name = 'ustawa'
        
        

Ustawa.init()


path="../data/ustawy"
i=0
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r',encoding='utf-8') as f: # open in readonly mode
        
        # title = 
        year = int(re.search(r'.*\/([^\/]{4})',f.name).group(1))
        first = Ustawa(title=f.name,content=f.read(),created_at = year)
        
        first.save(using=es)
        i+=1
        # if i==50:
        #     break
        # break
print(i) 








1179


In [594]:
from elasticsearch_dsl import *
from elasticsearch import *

In [599]:
from elasticsearch import MatchPhrase


ImportError: cannot import name 'MatchPhrase' from 'elasticsearch' (/home/x/anaconda3/envs/ml/lib/python3.8/site-packages/elasticsearch/__init__.py)

In [542]:
elasticsearch_dsl.connections.add_connection('python_client', es)

print(es.cat.plugins(params={'v': '', 'h': 'name,component,version,description'}))


name   component           version description
x-dell analysis-morfologik 7.10.1  Morfologik Polish Lemmatizer plugin for Elasticsearch



In [698]:
search = Search(index='ustawa',using=es).query('match', content='ustawa').count()
print(search)
# r =search.execute()
# print(len(r))

2356


In [547]:

s = Search(using=es, index="ustawa").count()
print(s)

1179


In [334]:
s = Search(using=es, index="ustawa").query("match_phrase", title='../data/ustawy/2001_906.txt').count()
print(s)

1


4. Define an ES index for storing the contents of the legislative acts.
5. Load the data to the ES index.
6. Determine the number of legislative acts containing the word **ustawa** (in any form).

In [696]:
s = Search(using=es, index="ustawa").filter("match_phrase", content='ustawa').count()
print(f'There are {s} documents with the word "ustawa" in them')

There are 2356 documents with the word "ustawa" in them


7. Determine the number of occurrences of the word **ustawa** by searching for this particular form, including the other inflectional forms.
8. Determine the number of occurrences of the word **ustaw** by searching for this particular form, including the other inflectional forms.

In [804]:
s = es.termvectors(index="ustawa", fields="content", id =id,field_statistics=False)
print(list(s['term_vectors']['content']['terms'].keys()))

print(s['term_vectors']['content']['terms']['ustawa']['term_freq'])

['1', '10a', '1989', '1990', '1993', '1996', '2', '20', '213', '3', '30', '306', '347', '435', '47', '53', '72', '73', 'artykuł', 'brzmienie', 'brzmieć', 'budżetowy', 'być', 'członek', 'dodawać', 'droga', 'dz.u', 'działalność', 'dzień', 'finansować', 'grudzień', 'i', 'kancelaria', 'kierowniczy', 'krajowy', 'który', 'maj', 'maja', 'moc', 'mowa', 'nie', 'numer', 'o', 'ocean', 'od', 'oda', 'ogłosić', 'ogłoszenie', 'ojciec', 'określać', 'on', 'państwowy', 'polski', 'pozycja', 'prezydent', 'przysługiwać', 'przyznawać', 'rad', 'rada', 'rok', 'ryczałt', 'rzeczpospolita', 'się', 'stanowisko', 'styczeń', 'sądownictwo', 'usta', 'ustawa', 'w', 'wchodzić', 'wiek', 'wysokość', 'z', 'zajmować', 'zajmujący', 'zarządzenie', 'zarządzić', 'zasada', 'zmiana', 'środek', 'środki', 'życie', 'żyto', 'żyć']
4


In [809]:
s = Search(using=es, index="ustawa").query("match_all")[:3000].execute()
ids = [x.meta.id for x in s]

amount=0
for id in ids:
    s = es.termvectors(index="ustawa", fields="content", id =id,field_statistics=True)
    try:
        amount += s['term_vectors']['content']['terms']['ustawa']['term_freq']
    except KeyError:
        pass
print(amount)

49868


9. Determine the number of legislative acts containing the words **kodeks postępowania cywilnego** 
   in the specified order, but in any inflection form.

In [766]:
s = Search(using=es, index="ustawa").query("match_phrase", content='kodeks postępowania cywilnego').count()
print(f'The number of files containing the words "kodeks postępowania cywilnego" is: {s}')

s = Search(using=es, index="ustawa").query("match_phrase", content='kpc').count()
print(f'The number of files containing the words "kpc" is: {s}')

The number of files containing the words "kodeks postępowania cywilnego" is: 198
The number of files containing the words "kpc" is: 198


10. Determine the number of legislative acts containing the words **wchodzi w życie** 
   (in any form) allowing for up to 2 additional words in the searched phrase.

In [631]:
q = elasticsearch_dsl.query.MatchPhrase(content={'query' : "wchodzi w życie",'slop' :2})
s = Search(using=es, index="ustawa").query(q).count()

print(s)

2348


11. Determine the 10 documents that are the most relevant for the phrase **konstytucja**.
12. Print the excerpts containing the word **konstytucja** (up to three excerpts per document) 
   from the previous task.

In [820]:
from pprint import pprint

s = Search(using=es, index="ustawa").filter("match_phrase", content='konstytucja').execute()
# s.highlight(fields='content').execute()
for i,hit in enumerate(s[:10]):
    # pprint(vars(hit))

    print(i, hit.title,hit.meta.score)
# print(s[:10])

0 ../data/ustawy/1995_436.txt 0.0
1 ../data/ustawy/2001_1082.txt 0.0
2 ../data/ustawy/1997_349.txt 0.0
3 ../data/ustawy/1996_396.txt 0.0
4 ../data/ustawy/1997_944.txt 0.0
5 ../data/ustawy/2001_23.txt 0.0
6 ../data/ustawy/2000_1268.txt 0.0
7 ../data/ustawy/1994_195.txt 0.0
8 ../data/ustawy/2002_110.txt 0.0
9 ../data/ustawy/1997_604.txt 0.0


In [851]:
q = elasticsearch_dsl.query.Match(content={'query' : "konstytucja"})
s = Search(using=es, index="ustawa").query(q).highlight('content',number_of_fragments=3,type='plain',fragment_size=80)[:10].execute()

for i,hit in enumerate(s):
    fragments = hit.meta['highlight']['content']
    print(i,hit['title'],hit.meta.score)
    for fragment in fragments:
        print('-', fragment)


0 ../data/ustawy/1997_629.txt 6.8801036
- 
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej Polskiej
-  przygotowania i 
uchwalenia <em>Konstytucji</em> Rzeczypospolitej Polskiej (Dz.U. Nr 67, poz
-  przedstawienia
                Zgromadzeniu Narodowemu projektu nowej <em>Konstytucji</em>
1 ../data/ustawy/1997_629.txt 6.8801036
- 
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej Polskiej
-  przygotowania i 
uchwalenia <em>Konstytucji</em> Rzeczypospolitej Polskiej (Dz.U. Nr 67, poz
-  przedstawienia
                Zgromadzeniu Narodowemu projektu nowej <em>Konstytucji</em>
2 ../data/ustawy/2000_443.txt 6.674678
- . 89 ust.
  1 i art. 90 <em>Konstytucji</em> Rzeczypospolitej Polskiej, oraz inne umowy
- . 89
     ust. 1 lub art. 90 <em>Konstytucji</em> Rzeczypospolitej Polskiej, lub
   3
-  przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
     Rzeczypospolitej
3 ../data/ustawy/2000_443.txt 6.674678
- . 89 ust.
  1 i art. 90 <em>Konstytu

In [586]:
s = Search(using=es, index="ustawa").query('nested', query=Q('match', content='konstytucja'), inner_hits={"highlight": {"fields": {"content": {}}}})

for hit in s:
    for comment in hit.meta.inner_hits.content.hits:
        # not 100% sure about the path, but look into comment.meta
        for fragment in comment.meta.highlight['content']:
            print(fragment)


RequestError: RequestError(400, 'illegal_argument_exception', "[nested] requires 'path' field")

In [454]:
s = Search().query('nested', path='comments', query=Q('match_phrase', content='konstytucja'), inner_hits={"highlight": {"fields": {"comments.content": {}}}})

for hit in s:
    for comment in hit.meta.inner_hits.comments.hits:
        # not 100% sure about the path, but look into comment.meta
        for fragment in comment.meta.highlight['comment.content']:
            print(fragment)


[]
